In [23]:
import torch

s_data = torch.randn(64,3,10)

In [24]:
QWs = s_data.split(2, dim = -1)

In [34]:
QWs = torch.cat(QWs, dim = 0)

In [27]:
mask = torch.randn(64,3, 2)

# torch.cat([mask for _ in range(10)], dim = 0)

In [28]:
soft = torch.nn.Softmax(dim=-1)

In [29]:
soft(mask)[0,0,:]

tensor([0.4086, 0.5914])

In [41]:
torch.cat(QWs.split(64, dim = 0), dim = -1).shape

torch.Size([64, 3, 10])

In [40]:
QWs.shape

torch.Size([320, 3, 2])

# transformer.py, _generate_mask, mask.unsqueeze(1).expand(*x.size(), mask.size(-1))

In [56]:
a = torch.randn(4)
b = torch.randn(3)
c = torch.randn(2)

In [57]:
kk = torch.zeros(3,4)
kk[0,:len(a)] = a
kk[1,:len(b)] = b
kk[2,:len(c)] = c

In [62]:
@torch.no_grad()
def _generate_mask(x, length):
    '''

    에) length : [4,3,2]
    '''
    mask = []

    max_length = max(length)
    for l in length:
        if max_length - l > 0:
            # If the length is shorter than maximum length among samples,
            # set last few values to be 1s to remove attention weight.
            mask += [torch.cat([x.new_ones(1, l).zero_(),
                                x.new_ones(1, (max_length - l))
                                ], dim=-1)]
        else:
            # If length of sample equals to maximum length among samples,
            # set every value in mask to be 0.
            mask += [x.new_ones(1, l).zero_()]

    mask = torch.cat(mask, dim=0).bool()
    # |mask| = (batch_size, max_length)

    return mask

In [63]:
mask = _generate_mask(kk, [4,3,2])

In [77]:
mask

tensor([[False, False, False, False],
        [False, False, False,  True],
        [False, False,  True,  True]])

In [74]:
mask.shape

torch.Size([3, 4])

In [78]:
kk.size(), mask.size(-1)

(torch.Size([3, 4]), 4)

In [71]:
mask.unsqueeze(1).expand(*kk.size(), mask.size(-1))

tensor([[[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]],

        [[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]]])

In [72]:
mask.unsqueeze(1)

tensor([[[False, False, False, False]],

        [[False, False, False,  True]],

        [[False, False,  True,  True]]])